In [ ]:
#!pip install category_encoders

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as st
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer, LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler 
from category_encoders.target_encoder import TargetEncoder

from sklearn.model_selection import cross_val_score
import seaborn as sns
from sklearn.model_selection import GridSearchCV

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Загрузка очищенных данных

In [5]:
df = pd.read_csv('final_data_new.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(df.shape)
df.head()

(4666, 93)


,t вып-обр,t обработка,t под током,t продувка,чист расход C,чист расход Cr,чист расход Mn,чист расход Si,температура первая,температура последняя,Ar (интенс.),эл. энергия (интенс.),произв жидкая сталь,произв количество обработок,расход газ Ar,сыпуч кокс пыль УСТК,сыпуч кокс. мелочь КМ1,ферспл CaC2,ферспл Mn5Si65Al0.5,ферспл SiMn18,ферспл фх850А,эл. энергия,химсталь первый Al_1,химсталь первый C_1,химсталь первый Cr_1,химсталь первый Cu_1,химсталь первый Mn_1,химсталь первый Mo_1,химсталь первый N_1,химсталь первый Ni_1,химсталь первый P_1,химсталь первый S_1,химсталь первый Si_1,химсталь первый Ti_1,химсталь первый V_1,химсталь последний Al,химсталь последний C,химсталь последний Ca,химсталь последний Cr,химсталь последний Cu,...,химшлак первый R_1,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2,химшлак последний Al2O3,химшлак первый Al2O3_1,136 RE,54E1,60E1,UIC54,НЕТ,ОР50,ОР65,Р50,Р65,Рельсы трамвайные,Шар заг d шара 100 мм 2 гр.,"Шар заг d шара 80-100 мм 2,3 гр.",Шар заг d шара 80-100 мм 4 гр.,Шары мелющие d 80-100 мм,Швеллер 40У,900А,G2HH,R260,R350HT,НХ,С255,Ш2.2,Ш2Л,Э73Ф,Э76,Э76Ф,Э76ХАФ,Э76ХФ,Э90ХАФ,сыпуч известь РП,химшлак первый SiO2_1
0,43.0,44.166667,21.683333,44.166667,0.481645,0.196138,0.201184,0.251281,1563.0,1588.0,14.163539,13769.100692,113.9,1.0,20.545,0.10,0.464,0.03,0.303,0.307,0.349000,4976.0,0.001,0.450,0.524,0.110,0.223,0.008,0.011,0.151,0.008,0.030,0.382,0.002,0.002,0.003,0.832,0.00050,0.692,0.110,...,1.8,56.2,0.4,7.8,0.07,2.2,26.0,5.7,3.8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.379,26.8
1,18.0,42.016667,20.016667,41.850000,0.339285,0.075060,0.066049,0.068946,1577.0,1590.0,8.060323,12550.541216,110.8,1.0,11.224,0.05,0.359,0.02,0.079,0.101,0.135000,4187.0,0.001,0.525,0.350,0.119,0.740,0.008,0.008,0.080,0.011,0.024,0.519,0.002,0.003,0.003,0.769,0.00032,0.415,0.121,...,2.3,54.8,0.8,6.1,0.09,2.4,23.2,3.2,2.7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,25.6
2,9.0,43.850000,18.950000,43.850000,0.433960,0.117998,0.175658,0.089606,1586.0,1596.0,10.810282,12012.664908,113.4,1.0,15.771,0.05,0.486,0.02,0.066,0.265,0.208581,3794.0,0.001,0.425,0.071,0.090,0.736,0.005,0.010,0.058,0.011,0.019,0.245,0.001,0.002,0.002,0.755,0.00020,0.071,0.090,...,2.4,54.8,0.6,11.6,0.16,2.1,26.1,4.7,4.8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,20.4
3,7.0,46.716667,19.216667,46.716667,0.548870,0.117998,0.199137,0.087354,1607.0,1594.0,9.288334,11908.412836,115.9,1.0,14.464,0.05,0.627,0.02,0.054,0.300,0.208581,3814.0,0.001,0.348,0.065,0.096,0.689,0.005,0.010,0.062,0.008,0.021,0.226,0.001,0.001,0.003,0.770,0.00020,0.066,0.095,...,2.3,51.9,0.5,10.3,0.15,2.1,24.2,5.4,4.9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,23.8
4,26.0,42.050000,19.883333,42.050000,0.402655,0.168487,0.176548,0.153988,1565.0,1588.0,18.348157,13609.388097,116.9,1.0,25.718,0.05,0.417,0.02,0.164,0.266,0.289000,4510.0,0.002,0.452,0.280,0.096,0.642,0.006,0.009,0.065,0.007,0.018,0.434,0.002,0.002,0.003,0.771,0.00030,0.419,0.097,...,2.2,54.5,0.3,14.0,0.14,2.1,26.6,5.2,5.3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN,22.4


In [6]:
data = df[df['сыпуч известь РП'].notnull()]
data.shape

(3712, 93)

## Тестирование моделей

In [7]:
def encode_categorial(name_scaler, scaler, 
                      data, y = None, 
                      return_scaler = False) -> list:

    '''
    Функция принимает название, обект и категориальные колонки данных
    Функция воврщает нормализованные данные и объект нормалайзера
    '''

    if name_scaler != 'TargetEncoder':
        if not return_scaler:
            return data[categorical_cols].apply(lambda x: scaler.fit_transform(x.astype(str)))
        else:
            return data[categorical_cols].apply(lambda x: scaler.fit_transform(x.astype(str))), scaler
    else:
        if not return_scaler:
            return scaler.fit_transform(data[categorical_cols], y)
        else:
            return scaler.fit_transform(data[categorical_cols], y), scaler
    
def encode_number(name_scaler, scaler, data, return_scaler = False) -> list:
  '''
    Функция принимает название, обект и числовые колонки данных
    Функция воврщает нормализованные данные и объект нормалайзера
    '''
    
    if name_scaler == 'Not scaled':
        if not return_scaler:
            return data
        else:
            return data, scaler
    else:
        if not return_scaler:
            return scaler.fit_transform(data)
        else:
            return scaler.fit_transform(data), scaler

In [8]:
def output_model(name_model, X_train, y_train, score = 'r2'):
  '''
    Функция принимает название, метрику оценки модели и колонки данных для подбора гиперпараметров
    Функция воврщает лучший объект модели
    '''
      
  if name_model=='GradientBoostingRegressor':
      parameters = {'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
                      'max_depth': [80, 90, 100, 110],
                      'max_features': [2, 3],
                      'min_samples_leaf': [3, 4, 5],
                      'min_samples_split': [8, 10, 12]}
        
      model = GradientBoostingRegressor(n_estimators=100, random_state=42)
      tree_grid = GridSearchCV(model, parameters,  cv=5, verbose=True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train)
      print(name_model)
      return tree_grid.best_estimator_
    
  elif name_model=='RandomForestRegressor':
      parameters = {'max_features': [2, 4, 6], 
                      'min_samples_leaf': [1, 3, 5, 10], 
                      'max_depth': [15,17,20,22,25]}
        
      model = RandomForestRegressor(n_estimators=100, random_state=42, 
                             oob_score=True)
      tree_grid = GridSearchCV(model, parameters, cv=5, verbose=True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train) 
      print(name_model)
      return tree_grid.best_estimator_
    
  elif name_model=='LassoCV':
      parameters = {'n_alphas': [50,100,150,200],
                      'max_iter': range(500,1500,100),
                     'selection':['cyclic', 'random']}
        
      model = LassoCV(random_state=42)
      tree_grid = GridSearchCV(model, parameters, cv=5,verbose = True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train) 
      print(name_model)
      return tree_grid.best_estimator_
    
  elif name_model=='RidgeCV':
      parameters = {'alpha': [1,0.1,0.01,0.001,0.0001,0] , 
                      "fit_intercept": [True, False],
                      "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
        
      model = RidgeCV(random_state=42)
      tree_grid = GridSearchCV(model, parameters, cv=5, verbose= True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train) 
      print(name_model)
      return tree_grid.best_estimator_
    
  elif name_model=='ElasticNetCV':
      parameters = {"max_iter": [1, 5, 10],
                      "n_alphas": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)}
        
      model = ElasticNetCV(random_state=42)
      tree_grid = GridSearchCV(model, parameters,  cv=5, verbose=True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train) 
      print(name_model)
      return tree_grid.best_estimator_
    
  elif name_model=='ExtraTreesRegressor':
      parameters={ 'max_features': ['auto', 'sqrt', 'log2'],
                     'min_samples_leaf': range(20,50,5),
                     'min_samples_split': range(15,36,5)}
        
      model = ExtraTreesRegressor(n_estimators=100, random_state=42, 
                                    bootstrap=True, oob_score=True)
      tree_grid = GridSearchCV(model, parameters,  cv=5, verbose=True,
                               n_jobs= -1, scoring = score)
      tree_grid.fit(X_train, y_train) 
      print(name_model)
      return tree_grid.best_estimator_

  else:

      model = LinearRegression()
      model.fit(X_train, y_train)
      print(name_model)
      return model

In [ ]:
mls = {
        'GradientBoostingRegressor': GradientBoostingRegressor(random_state=0), 
         'RandomForestRegressor':RandomForestRegressor(max_depth=2, random_state=0),
        'LassoCV': LassoCV(), 'RidgeCV': RidgeCV(), 'ElasticNetCV': ElasticNetCV(),
       'ExtraTreesRegressor': ExtraTreesRegressor(n_estimators=100, random_state=0)
        }

scalers = {'StandardScaler': StandardScaler()}

mlss = []
metrics = ['r2', 'neg_mean_absolute_error', 'neg_mean_squared_error']

X = data.drop(['сыпуч известь РП'], axis = 1)
y = data['сыпуч известь РП'].values

for idd, (name_ml, ml) in enumerate(mls.items()):
  print(name_ml)
  data_new = pd.DataFrame()
        
  for name_scaler, scaler in scalers.items():
      data_new = encode_number(name_scaler, scaler, X)

      for metric in metrics:
        X_train, X_test, y_train, y_test = train_test_split(data_new, y, test_size=0.33, random_state=42, shuffle = True)
        model = output_model(name_ml, X_train, y_train, score= metric)
        
        mlss.append([name_ml, model, metric, name_scaler, round(model.score(X_train, y_train),3),
                     round(model.score(X_test, y_test),3),
                     cross_val_score(model, X_test, y_test, cv=5)])